# **REVERSE DIFFUSION**
**License : CC BY-NC-SA 4.0**

In this notebook, we demonstrate how to train our ddpm model. 
Source :  
- https://huggingface.co/blog/annotated-diffusion  
- https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=5153024b  

## **Jean Zay Environment**
We should use the `pytorch-gpu-2.1.1_py3.11.5` jupyter kernel.

In [ ]:
!module list

## **Import libraries**

In [ ]:
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.optim import Adam

from code.model import Unet
from code.fashion_mnist_dataset import get_dataloader
from code.forward_diffusion import linear_beta_schedule, cosine_beta_schedule, get_alph_bet, q_sample
from code.reverse_diffusion import train, sampling
from code.utils import show_images, make_gif

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
DEVICE = "cpu"

## **Model definition**
The reverse diffusion process is made with the help of a deep learning model. We choosed a Unet model with attention. The model is optimized following some papers like [ConvNeXt](https://arxiv.org/pdf/2201.03545.pdf). You can inspect the model in `code/model.py` file.

In [ ]:
model = Unet(   
    dim=28,
    init_dim=None,
    out_dim=None,
    dim_mults=(1, 2, 4),
    channels=1,
    with_time_emb=True,
    convnext_mult=2,
).to(DEVICE)

In [ ]:
# CHECK IF OUR model IS IN GPU
print(f"Is our model in GPU ? {next(model.parameters()).is_cuda}")

## **Training loop**
![training_algorithm](img/training_algorithm.png)

In [ ]:
# ARGUMENTS
epochs = 3
T = 1000
batch_size = 64

# GET DATALOADER
train_dataloader, channels, image_size, n_batch = get_dataloader(batch_size=batch_size,
                                                                 test=False)

# GET BETA SCHEDULE
constants_dict = get_alph_bet(T, schedule=linear_beta_schedule)

### **Generate image from untrained model**

In [ ]:
print("Check generation :")  
list_gen_imgs = sampling(model, (batch_size, channels, image_size, image_size), T, constants_dict)
show_images(list_gen_imgs[-1])

### **TODO**
**Complete the training loop to train the model**  
Solution : `train` in `reverse_diffusion.py` 

In [ ]:
# model = train(epochs, train_dataloader, model, T, constants_dict)

In [ ]:
criterion = nn.SmoothL1Loss()
optimizer = Adam(model.parameters(), lr=1e-4)

for epoch in range(epochs):
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")
    for batch in loop:
        optimizer.zero_grad()

        batch_size_iter = batch["pixel_values"].shape[0]
        batch_image = batch["pixel_values"].to(DEVICE)

        # GENERATE NOISE LEVEL FOR EACH IMAGE
        batch_t = None # don't forget to instantiate batch_t on the same device as model
        # GENERATE GAUSSIAN NOISE N(0,1)
        noise = None # don't forget to instantiate batch_t on the same device as model
        # CREATE NOISY IMAGE
        x_noisy = None # don't forget to instantiate batch_t on the same device as model
        # PREDICT NOISE WITH THE MODEL
        predicted_noise = None
        # COMPUTE LOSS
        loss = None
            
        loop.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()

### **Generate image after training model**

In [ ]:
print("Check generation:")  
list_gen_imgs = sampling(model, (batch_size, channels, image_size, image_size), T, constants_dict)
show_images(list_gen_imgs[-1])

In [ ]:
# VISUALISE denoising steps
make_gif(list_gen_imgs)

## **Let's play**
Now that we have a complete training procedure and a sampling procedure, we can test different configurations !

In [ ]:
"""
model = Unet(   
    dim=28,
    init_dim=None,
    out_dim=None,
    dim_mults=(1, 2, 4),
    channels=1,
    with_time_emb=True,
    convnext_mult=2,
).to(DEVICE)

constants_dict = get_alph_bet(T, schedule=linear_beta_schedule)
"""

# ARGUMENTS
epochs2 = 3
T2 = 1000

# GET MODEL
model2 = Unet(   
    dim=28,
    init_dim=None,
    out_dim=None,
    dim_mults=(1, 2, 4),
    channels=1,
    with_time_emb=True,
    convnext_mult=2,
).to(DEVICE)

# GET DATALOADER
train_dataloader2, channels2, image_size2, n_batch2 = get_dataloader(batch_size=64,
                                                                     test=False)
# GET BETA SCHEDULE
constants_dict2 = get_alph_bet(T2, schedule=cosine_beta_schedule)

# TRAIN MODEL
model2 = train(epochs2, train_dataloader2, model2, T2, constants_dict2)

In [ ]:
print("Check generation:")  
list_gen_imgs2 = sampling(model2, (batch_size, channels2, image_size2, image_size2), T2, constants_dict2)
show_images(list_gen_imgs2[-1])